In [1]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#from keybert import KeyBERT
import yake

In [2]:
sds_text = pd.read_csv('../data/sds_text.csv')

<IPython.core.display.Javascript object>

In [3]:
sds_text

,Unnamed: 0,episode_name,length_episode,context_episode,guest_name,guest_info,text_episode,episode_number,episode_date,episode_day,host_episode,speaker,episode_split_text
0,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,This is episode number one with ex-chemical e...
1,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,"Hey guys, welcome to the Podcast. I’ve got Ru..."
2,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,Thank you! Thanks for having me over. I’m doi...
3,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Kirill,Awesome. It’s great to hear you and for those...
4,0,"Ruben Kogel on Self-Serve Analytics, R vs Pyt...",42,Business Data Science Database,Ruben Kogel,"Subscribe on Website, Apple Podcasts, Spotify,...",Kirill: This is episode number one with ex-che...,1,"Sep 10, 2016",Saturday,Kirril Eremenko,Ruben,"Yeah sure. So, I’m the senior manager of anal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46767,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Yeah, right. Yeah, as I mean, it actually, it ..."
46768,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Happy to.
46769,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Jon Krohn,"Nice. Well, yeah, so you mentioned potentially..."
46770,681,"Contextual A.I. for Adapting to Adversaries, ...",81,Data Science Artificial Intelligence,Matar Haller,"Subscribe on Website, Apple Podcasts, Spotify,...",Jon Krohn:\t00:00:05\tThis is episode number 6...,683,"May 30, 2023",Tuesday,Jon Krohn,Matar Haller,Thank you for having me. This was fascinating ...


In [4]:
sds_text = sds_text.set_index('Unnamed: 0')

In [5]:
sds_processed = pd.DataFrame(sds_text.groupby(['episode_number', 'episode_name', 'length_episode', 'context_episode', 'guest_name', 'host_episode', 'episode_date'])['episode_split_text'].agg(lambda x: ' '.join(x))).reset_index()

<IPython.core.display.Javascript object>

In [ ]:
sds_processed

In [6]:
# Basic function to clean the text 
def clean_text(text):     
    return text.lower()

In [7]:
sds_processed['episode_split_text'] = sds_processed['episode_split_text'].apply(clean_text)

In [ ]:
sds_processed

In [8]:
sds_processed['episode_keywords'] = None

In [9]:
def yake_keyword_extractor(doc):
    global sds_processed
    
    for i, episode_text in doc.iteritems():
        max_ngram_size = 3
        deduplication_threshold = 0.3
        windowSize = 1
        numOfKeywords = 100
        kw_extractor = yake.KeywordExtractor(n = max_ngram_size, dedupLim = deduplication_threshold , windowsSize = windowSize, top = numOfKeywords)
        keywords = kw_extractor.extract_keywords(episode_text)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
yake_keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
sds_processed.to_csv('../data/sds_yake_keywords.csv')

In [ ]:
sds_keywords = pd.read_csv('../data/sds_yake_keywords.csv')

In [ ]:
sds_keywords

In [ ]:
word_tuples = [('data', 0.0010532908775940833), ('data scientist', 0.0029141325644050945), ('people', 0.005422867923900782), ('n’t', 0.006871265577956575), ('’re', 0.008884744001205683), ('analysis', 0.01281082680029214), ('udemy', 0.016638427011567425), ('thing', 0.018489819573940445), ('work', 0.026106167845171176), ('ruben', 0.028702291819461337), ('build', 0.03003168690508715), ('team', 0.032220440860286116), ('show', 0.040495101361532), ('question', 0.04063280878564204), ('make', 0.04448616608751784), ('important', 0.04568613147206454), ('insights', 0.050842033371277406), ('powerful', 0.05251763498491655), ('sql', 0.059737659259588516), ('listeners', 0.060455501278664454), ('red shift', 0.06452519251510098), ('background', 0.06521651854598134), ('sensitive data', 0.06798193640183485), ('elemental data problems', 0.06808923752012126), ('totally', 0.07980158442301752), ('end', 0.09462167394867813), ('communication skills', 0.09901294474504588), ('engineering', 0.1135042667193427), ('platform', 0.1139030943265721), ('statistical', 0.11869365178842255), ('influence', 0.11922058764723066), ('project', 0.12777923082182785), ('science wizard', 0.1361525255478297), ('convey', 0.1427370340547599), ('job', 0.14407934432290917), ('episode', 0.14483249042899463), ('n’t wanna work', 0.15254512892072472), ('problem solving thing', 0.16422008206512995), ('’ve learned', 0.16981820700395897), ('pretty cool', 0.17067999701657272), ('size', 0.1821297897355906), ('people who specialize', 0.18974036853129833), ('explain', 0.19046069125821452), ('ability', 0.19388868834551098), ('aws', 0.19481837195406812), ('articles', 0.21889215391276715), ('ruben kogel.welcome', 0.2383267121932403), ('strategy', 0.26597500290820864), ('leverage', 0.26943937566389226), ('push', 0.2706788376330779), ('check', 0.2745931680444767), ('fact', 0.27542786666192787), ('breiman', 0.27573806523021577), ('’re quickly', 0.2869003093342987), ('metric', 0.39239618738482207), ('discussed lots', 0.4224813160987914), ('jump', 0.42569932503668617), ('biggest', 0.43062024110563135), ('scalable', 0.4348002572368825), ('summary', 0.43507210437225546), ('half', 0.43566181067001614), ('view', 0.43632798500045983), ('advanced', 0.43821567421603597), ('trustworthy', 0.43914585044899046), ('behavior', 0.44014923866039657), ('ex-chemical engineer', 0.47417419541876416), ('yeah yeah', 0.47748801172975774), ('efficient analytics team', 0.49106273295984), ('techniques you ’re', 0.5568305258185692), ('translate a lot', 0.6642515176584556), ('stakeholders coming', 0.6709457645693998), ('lots of regression', 0.7080528578956833), ('noise by nate', 0.7367804020215138), ('thing in communication', 0.74777341131578), ('good idea', 0.8026798733239394), ('measuring student', 0.8130853605951406), ('off.', 0.8512416854015954), ('guys', 0.869753315936046), ('glad', 0.8775854997392524), ('blog', 0.8785492259628155), ('skills nevertheless. yeah', 0.8870029931682176), ('touched', 0.9085820192716929), ('self-served', 0.9085820192716929), ('width', 0.9201510701708244), ('director', 0.9205577391019704), ('one.the', 0.9235531647655699), ('fix', 0.9279796911765633), ('anymore', 0.9293875146481567), ('uhuh', 0.9299820716737279), ('prioritized', 0.93152190035003), ('graphics', 0.9323728330400206), ('occur', 0.9328737183606961), ('stuff', 0.9333168252017048), ('10-12', 0.9340346598369818), ('necessarily', 0.9353005575308527), ('divisions in udemy', 0.9430098537561858), ('analytics and predictive', 1.1000320353744777), ('complex technical', 1.1592371306994007), ('kirill eremenko', 1.1877887135345484), ('kicking things off.', 1.2619808102396495)]

words = [t[0] for t in word_tuples]

print(words)

In [ ]:
sds_1 = sds_keywords[sds_keywords['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
sds_1.info()

In [ ]:
sds_1['episode_keywords'].loc[0]

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = None

In [ ]:
def tuple_extractor(row):
    words = [t[0] for t in row]
    return words

In [ ]:
sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(word_tuples):
    global sds_1
    words = [t[0] for t in word_tuples]
    return words

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(lambda x: tuple_extractor(x))

In [ ]:
sds_1

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1['keywords'].loc[0]

In [ ]:
def tuple_extractor(tuple):
    global sds_keywords
    words = []
    #for i, episode_text in doc.iteritems():
    for i, word_tuple in tuple.iteritems():
        words = [t[0] for t in word_tuple]
        sds_keywords['keywords'] = words
    
            

In [ ]:
sds_keywords['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_keywords

In [ ]:
sds_1['keywords'] = sds_1['episode_keywords'].apply(tuple_extractor)

In [ ]:
sds_1

In [ ]:
[t[0] for t in sds_1['episode_keywords']]

In [ ]:
sds_processed['episode_keywords'] = [t[0] for t in sds_processed['episode_keywords']]

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_keywords'].loc[678]

In [ ]:
doca = sds_processed['episode_split_text'].loc[1]
docb = sds_processed['episode_split_text'].loc[150]
docc = sds_processed['episode_split_text'].loc[400]
docd = sds_processed['episode_split_text'].loc[679]

In [ ]:
def keyword_extractor(doc):
    kw_model = KeyBERT()
    global sds_processed
    stopwords = list(STOP_WORDS)
    
    for i, episode_text in doc.iteritems():
        keywords = kw_model.extract_keywords(episode_text, keyphrase_ngram_range=(1, 1), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.7)
        sds_processed.at[i, 'episode_keywords'] = keywords

In [ ]:
kw_model = KeyBERT()
stopwords = list(STOP_WORDS)
keywords = kw_model.extract_keywords(doca, keyphrase_ngram_range=(1, 2), stop_words = stopwords, top_n = 100, use_mmr=True, diversity=0.8)



In [ ]:
keywords

In [ ]:
keyword_extractor(sds_processed['episode_split_text'])

In [ ]:
len(keywords)

In [ ]:
keywords

In [ ]:
#sds_processed['episode_split_text'][1:5].apply(keyword_extractor)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html